# Classification of gestures 

### Imports 

In [ ]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader,Dataset
num_classes = 8  # no. of classes in your model
epochs = 1000  # training epochs
in_features = 63  # in_channels to the model 
device = "cuda" if torch.cuda.is_available() else "cpu"

### Making the dataset from csv file

In [ ]:
class make_dataset(Dataset):
  
    def __init__(self):
        super().__init__()
        data1 = np.loadtxt('csv/file/path.csv', delimiter=',',
                           dtype=np.float32)
        self.x = torch.from_numpy(data1[:, 1:])
        self.y = torch.from_numpy(data1[:, [0]]).type(torch.LongTensor)
        self.n_samples = data1.shape[0] 
      
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.n_samples

In [ ]:
data = make_dataset()
data[0]

### Making Dataloader from dataset 

In [ ]:
dataloader = DataLoader(data, batch_size=64, shuffle=True)

### Model

In [ ]:
model = nn.Sequential(
    nn.Linear(in_features=in_features,out_features=50),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(in_features=50,out_features=30),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(in_features=30, out_features=20),
    nn.ReLU(),
    nn.Linear(in_features=20,out_features=num_classes)
).to(device)

In [ ]:
from torchinfo import summary
""" To find the status of model uncomment below 4 lines"""
print(summary(model,
        input_size=(8192,63),
        col_names=['input_size','output_size','num_params','trainable'],
        col_width=18))

### Loss Function and Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr = 0.0001)

### Training Loop in Pytorch

In [ ]:
for epoch in range(epochs):
    acc = 0
    for i,(X, y) in enumerate(dataloader):
        model.train()
        X, y = X.to(device), y.to(device)
        logit = model(X)
        # print(X)
        # acc += (torch.argmax(logit,dim=1)==torch.flatten(y)).sum()/len(y)
        loss = loss_fn(logit, torch.flatten(y))
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model.eval()
        logit = model(X)
        acc += (torch.argmax(logit,dim=1)==torch.flatten(y)).sum()/len(y)
    print(acc/len(dataloader))
        

### Saving the model in pickle file

In [ ]:
# save the model 
import pickle
pickle.dump(model,open(f"model_{device}.pk","wb"))